0 · Prerequisites — edit the three paths below first

In [1]:
# ⬅️ CHANGE THESE
CHECKPOINT_DIR = "/rsrch1/ip/msalehjahromi/codes/FineTune/multiGPU/n_GPU_A100/output_ddp_p16_23_1150/checkpoints"

BASE_CKPT       = f"{CHECKPOINT_DIR}/base_model_epoch_18.pt"  #base_model_final.pt   # or base_model_epoch_X.pt
AGG_CKPT        = f"{CHECKPOINT_DIR}/aggregator_epoch_18.pt"  #aggregator_final.pt   # or aggregator_epoch_X.pt
CSV_PATH        = "/rsrch1/ip/msalehjahromi/codes/FineTune/nlst_event_train_val_test_.csv"


1 · Imports & helper functions

In [2]:
import os, json, torch, numpy as np, pandas as pd
from torch.utils.data import DataLoader
from sklearn.metrics import roc_auc_score

# add dinov2 + your util folder to sys.path
import sys
sys.path.insert(0, "/rsrch1/ip/msalehjahromi/codes/dinov2-torchrun-dataloader6")
sys.path.insert(0, "/rsrch1/ip/msalehjahromi/codes/FineTune")  # where model_utils.py lives

from dinov2.models.vision_transformer import DinoVisionTransformer  # base CNN/ViT
from model_utils import CombinedModel, VolumeProcessor, NLSTDataset   # your utilities


/rsrch1/ip/msalehjahromi/codes/dinov2-torchrun-dataloader6/dinov2/layers/swiglu_ffn.py:43: UserWarning: xFormers is available (SwiGLU)
  warnings.warn("xFormers is available (SwiGLU)")
/rsrch1/ip/msalehjahromi/codes/dinov2-torchrun-dataloader6/dinov2/layers/attention.py:27: UserWarning: xFormers is available (Attention)
  warnings.warn("xFormers is available (Attention)")
/rsrch1/ip/msalehjahromi/codes/dinov2-torchrun-dataloader6/dinov2/layers/block.py:33: UserWarning: xFormers is available (Block)
  warnings.warn("xFormers is available (Block)")
/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2 · Rebuild the base model exactly as in training

In [3]:
# ⚙️ MUST match the hyper-params used during training
base_model = DinoVisionTransformer(
    img_size        = 448,
    patch_size      = 16,
    drop_path_rate  = 0.0,
    block_chunks    = 1,
    drop_path_uniform=True,
    embed_dim       = 768,
    depth           = 12,
    num_heads       = 12,
    mlp_ratio       = 4,
    num_register_tokens = 5,
    init_values     = 1e-5,
)

# materialise tokens (same util used in training)
def _materialize_tokens(m):
    with torch.no_grad():
        for n in ["cls_token", "register_tokens", "mask_token"]:
            if hasattr(m, n) and getattr(m, n).storage().size() == 0:
                real = torch.zeros_like(getattr(m, n))
                torch.nn.init.normal_(real, std=0.02)
                setattr(m, n, torch.nn.Parameter(real, requires_grad=True))
_materialize_tokens(base_model)


/tmp/ipykernel_12894/3914724356.py:20: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  if hasattr(m, n) and getattr(m, n).storage().size() == 0:


3 · Build the CombinedModel (aggregator/MLP head)

In [4]:
NUM_TASKS = 6           # 6 cancer-risk horizons
model = CombinedModel(
    base_model      = base_model,
    chunk_feat_dim  = 768,
    hidden_dim      = 1024,
    num_tasks       = NUM_TASKS,
    num_attn_heads  = 3,        # ⬅️ match your --num-attn-heads
    num_layers      = 2,        # ⬅️ match your --num-layers
    dropout_rate    = 0.35,      # ⬅️ match your --dropout
)


4 · Load the weights
(If you instead have one combined checkpoint called model_final.pt, just replace 4-A and 4-B with model.load_state_dict(torch.load("model_final.pt")).)

In [5]:
# 4-A  load base backbone
base_state = torch.load(BASE_CKPT, map_location="cpu")
missing, unexpected = model.base.load_state_dict(base_state, strict=False)
print("Base model  – missing:", len(missing), " unexpected:", len(unexpected))

# 4-B  load the aggregator / transformer + classifier
agg_state  = torch.load(AGG_CKPT,  map_location="cpu")
# keys were saved without 'base.' prefix, so we can load directly
missing, unexpected = model.load_state_dict(agg_state, strict=False)
print("Aggregator – missing:", len(missing), " unexpected:", len(unexpected))


Base model  – missing: 0  unexpected: 0
Aggregator – missing: 176  unexpected: 0


5 · Send model to device & eval-mode

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device).eval()
print(device)

cuda:0


6 · Prepare the test split

In [7]:
label_cols = ['1-year-cancer','2-year-cancer','3-year-cancer',
              '4-year-cancer','5-year-cancer','6-year-cancer']

test_df = (pd.read_csv(CSV_PATH)
             .query("split == 'test' and file_path.notna()")
             .reset_index(drop=True))
print("Test patients:", len(test_df))

# same preprocessing params as training
processor = VolumeProcessor(chunk_depth=3, out_size=(448,448))

test_ds = NLSTDataset(test_df, processor, label_cols, augment=None)
test_dl = DataLoader(test_ds,
                     batch_size=1,          # one patient at a time
                     shuffle=False,
                     num_workers=4,
                     collate_fn=lambda b: b[0])

test_df.head(1)

Test patients: 5136


,pid,gender,age,pkyr,cigsmok,diagcopd,famfather,scr_res0,scr_res1,scr_res2,...,time,time_years,1-year-cancer,2-year-cancer,3-year-cancer,4-year-cancer,5-year-cancer,6-year-cancer,split,file_path
0,121403_T0,1,56,54.0,1,0.0,0.0,2,1,3,...,2001.0,5.478439,0,0,0,0,0,-1,test,/rsrch7/home/ip_rsrch/wulab/Lung_Foundation_Mo...


7 · Utility: quick evaluate() for single-GPU inference

In [8]:
# @torch.no_grad()
# def evaluate_verbose(model, loader, df_meta, device,
#                      save_csv="test_predictions.csv",
#                      print_each=True):          # ← new flag
#     model.eval()
#     records = []

#     for row_idx, (chunks, labels, mask, spacing) in enumerate(loader):
#         pid = df_meta.loc[row_idx, "pid"] if "pid" in df_meta.columns else row_idx
        
#         chunks = chunks.to(device)
#         logits = model(chunks, spacing).squeeze(0)      # [T]
#         probs  = torch.sigmoid(logits)

#         rec = {"row": row_idx, "patient_id": pid}
#         for t in range(logits.numel()):
#             rec[f"logit_t{t}"]  = logits[t].item()
#             rec[f"prob_t{t}"]   = probs[t].item()
#             rec[f"target_t{t}"] = float(labels[t])
#             rec[f"mask_t{t}"]   = bool(mask[t])
#         records.append(rec)

#         # 🔸 real-time print of this patient
#         if print_each:
#             summary = " | ".join(
#                 f"T{t}: {probs[t]:.3f} (y={int(labels[t])})"
#                 for t in range(logits.numel()) if mask[t]
#             )
#             # spmehow show accuracy AUC using the previous n patients
#             print(f"[{row_idx}] {pid} {chunks.shape} → {summary}")
#             print()

#     out_df = pd.DataFrame(records)
#     out_df.to_csv(save_csv, index=False)
#     print(f"\nSaved detailed predictions to {save_csv}")
#     display(out_df.head(10))           # still show a quick peek
#     return out_df

@torch.no_grad()
def evaluate_verbose(model, loader, df_meta, device,
                     save_csv="test_predictions.csv",
                     print_each=True):
    model.eval()
    recs = []

    # running containers for metrics
    all_probs, all_labels, all_masks = [], [], []

    for idx, (chunks, labels, mask, spacing) in enumerate(loader):
        pid = df_meta.loc[idx, "pid"] if "pid" in df_meta.columns else idx

        chunks = chunks.to(device)
        logits = model(chunks, spacing).squeeze(0)          # [T]
        probs  = torch.sigmoid(logits).cpu().numpy()        # [T]

        rec = {"row": idx, "patient_id": pid}
        for t in range(logits.numel()):
            rec[f"logit_t{t}"]  = logits[t].item()
            rec[f"prob_t{t}"]   = probs[t].item()
            rec[f"target_t{t}"] = float(labels[t])
            rec[f"mask_t{t}"]   = bool(mask[t])
        recs.append(rec)

        # ╭─────── update running lists ───────╮
        all_probs.append(probs)
        all_labels.append(labels)
        all_masks.append(mask)
        # ╰─────────────────────────────────────╯

        if print_each:
            summary = " | ".join(
                f"T{t}:{probs[t]:.3f}(y={int(labels[t])})"
                for t in range(len(probs)) if mask[t]
            )

            # compute running accuracy / auc
            import numpy as np
            from sklearn.metrics import roc_auc_score

            sofar_probs   = np.vstack(all_probs)
            sofar_targets = np.vstack(all_labels)
            sofar_masks   = np.vstack(all_masks)

            metrics_str = []
            for t in range(sofar_probs.shape[1]):
                valid = sofar_masks[:, t].astype(bool)
                if valid.sum() == 0:
                    continue

                # accuracy (threshold 0.5)
                acc = ( (sofar_probs[valid, t] > 0.5) ==
                        sofar_targets[valid, t] ).mean()

                # auc (only if we have both classes)
                try:
                    auc = roc_auc_score(sofar_targets[valid, t], sofar_probs[valid, t])
                    auc_txt = f"{auc:.3f}"
                except ValueError:
                    auc_txt = "–"

                metrics_str.append(f"T{t} acc={acc:.2f} auc={auc_txt}")

            print(f"[{idx}] {pid} {chunks.shape} → {summary}")
            print("   running:", "; ".join(metrics_str), "\n")

    # save CSV and return DF (same as before) …
    out_df = pd.DataFrame(recs)
    out_df.to_csv(save_csv, index=False)
    print(f"\nSaved detailed predictions to {save_csv}")
    display(out_df.head(10))
    return out_df


8 · Run the evaluation

In [9]:
# metrics, logits, targets, masks = evaluate(model, test_dl, device)
# print(json.dumps(metrics, indent=2))
detailed_df = evaluate_verbose(model, test_dl, test_df, device)








/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(


[0] 121403_T0 torch.Size([39, 3, 448, 448]) → T0:0.008(y=0) | T1:0.014(y=0) | T2:0.019(y=0) | T3:0.025(y=0) | T4:0.021(y=0)
   running: T0 acc=1.00 auc=nan; T1 acc=1.00 auc=nan; T2 acc=1.00 auc=nan; T3 acc=1.00 auc=nan; T4 acc=1.00 auc=nan 



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(


[1] 203699_T0 torch.Size([41, 3, 448, 448]) → T0:0.010(y=0) | T1:0.015(y=0) | T2:0.020(y=0) | T3:0.027(y=0) | T4:0.022(y=0)
   running: T0 acc=1.00 auc=nan; T1 acc=1.00 auc=nan; T2 acc=1.00 auc=nan; T3 acc=1.00 auc=nan; T4 acc=1.00 auc=nan 



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(


...

[5034] 213994_T2 torch.Size([72, 3, 448, 448]) → T0:0.013(y=0) | T1:0.018(y=0) | T2:0.028(y=0) | T3:0.036(y=0) | T4:0.029(y=0)
   running: T0 acc=0.99 auc=0.638; T1 acc=0.98 auc=0.671; T2 acc=0.97 auc=0.644; T3 acc=0.96 auc=0.665; T4 acc=0.93 auc=0.668; T5 acc=0.88 auc=0.687 

[5035] 213999_T2 torch.Size([71, 3, 448, 448]) → T0:0.015(y=0) | T1:0.029(y=0) | T2:0.037(y=0) | T3:0.049(y=0)
   running: T0 acc=0.99 auc=0.638; T1 acc=0.98 auc=0.671; T2 acc=0.97 auc=0.644; T3 acc=0.96 auc=0.665; T4 acc=0.93 auc=0.668; T5 acc=0.88 auc=0.687 

[5036] 214011_T2 torch.Size([43, 3, 448, 448]) → T0:0.005(y=0) | T1:0.008(y=0) | T2:0.011(y=0) | T3:0.012(y=0)
   running: T0 acc=0.99 auc=0.638; T1 acc=0.98 auc=0.671; T2 acc=0.97 auc=0.644; T3 acc=0.96 auc=0.665; T4 acc=0.93 auc=0.668; T5 acc=0.88 auc=0.687 

[5037] 214058_T2 torch.Size([68, 3, 448, 448]) → T0:0.030(y=0) | T1:0.053(y=0) | T2:0.074(y=0) | T3:0.105(y=0) | T4:0.097(y=0)
   running: T0 acc=0.99 auc=0.638; T1 acc=0.98 auc=0.671; T2 acc=0.97 a

[5066] 215873_T2 torch.Size([55, 3, 448, 448]) → T0:0.015(y=0) | T1:0.024(y=0) | T2:0.032(y=0) | T3:0.044(y=0)
   running: T0 acc=0.99 auc=0.637; T1 acc=0.98 auc=0.670; T2 acc=0.97 auc=0.643; T3 acc=0.96 auc=0.664; T4 acc=0.93 auc=0.668; T5 acc=0.88 auc=0.687 

[5067] 215876_T2 torch.Size([54, 3, 448, 448]) → T0:0.010(y=0) | T1:0.015(y=0) | T2:0.022(y=0) | T3:0.028(y=0) | T4:0.028(y=0)
   running: T0 acc=0.99 auc=0.637; T1 acc=0.98 auc=0.670; T2 acc=0.97 auc=0.643; T3 acc=0.96 auc=0.664; T4 acc=0.93 auc=0.668; T5 acc=0.88 auc=0.687 

[5068] 215887_T2 torch.Size([74, 3, 448, 448]) → T0:0.010(y=0) | T1:0.012(y=0) | T2:0.021(y=0) | T3:0.021(y=0)
   running: T0 acc=0.99 auc=0.637; T1 acc=0.98 auc=0.670; T2 acc=0.97 auc=0.643; T3 acc=0.96 auc=0.664; T4 acc=0.93 auc=0.668; T5 acc=0.88 auc=0.687 

[5069] 215938_T2 torch.Size([41, 3, 448, 448]) → T0:0.008(y=0) | T1:0.014(y=0) | T2:0.017(y=0) | T3:0.023(y=0)
   running: T0 acc=0.99 auc=0.637; T1 acc=0.98 auc=0.670; T2 acc=0.97 auc=0.643; T3 acc

[5097] 216905_T2 torch.Size([51, 3, 448, 448]) → T0:0.019(y=0) | T1:0.028(y=0) | T2:0.044(y=0) | T3:0.054(y=0) | T4:0.040(y=0)
   running: T0 acc=0.99 auc=0.636; T1 acc=0.98 auc=0.670; T2 acc=0.97 auc=0.643; T3 acc=0.96 auc=0.664; T4 acc=0.93 auc=0.667; T5 acc=0.88 auc=0.687 

[5098] 217038_T2 torch.Size([38, 3, 448, 448]) → T0:0.005(y=0) | T1:0.007(y=0) | T2:0.011(y=0) | T3:0.013(y=0)
   running: T0 acc=0.99 auc=0.636; T1 acc=0.98 auc=0.670; T2 acc=0.97 auc=0.643; T3 acc=0.96 auc=0.664; T4 acc=0.93 auc=0.667; T5 acc=0.88 auc=0.687 

[5099] 217053_T2 torch.Size([51, 3, 448, 448]) → T0:0.006(y=0) | T1:0.009(y=0) | T2:0.014(y=0) | T3:0.018(y=0)
   running: T0 acc=0.99 auc=0.636; T1 acc=0.98 auc=0.670; T2 acc=0.97 auc=0.643; T3 acc=0.96 auc=0.664; T4 acc=0.93 auc=0.667; T5 acc=0.88 auc=0.687 

[5100] 217084_T2 torch.Size([36, 3, 448, 448]) → T0:0.006(y=0) | T1:0.008(y=0) | T2:0.012(y=0) | T3:0.013(y=0) | T4:0.013(y=0)
   running: T0 acc=0.99 auc=0.636; T1 acc=0.98 auc=0.670; T2 acc=0.97 a

[5129] 218611_T2 torch.Size([45, 3, 448, 448]) → T0:0.007(y=0) | T1:0.009(y=0) | T2:0.013(y=0) | T3:0.015(y=0)
   running: T0 acc=0.99 auc=0.635; T1 acc=0.98 auc=0.669; T2 acc=0.97 auc=0.642; T3 acc=0.96 auc=0.662; T4 acc=0.93 auc=0.667; T5 acc=0.88 auc=0.687 

[5130] 218618_T2 torch.Size([47, 3, 448, 448]) → T0:0.014(y=0) | T1:0.027(y=0) | T2:0.030(y=0) | T3:0.049(y=0)
   running: T0 acc=0.99 auc=0.635; T1 acc=0.98 auc=0.669; T2 acc=0.97 auc=0.641; T3 acc=0.96 auc=0.662; T4 acc=0.93 auc=0.667; T5 acc=0.88 auc=0.687 

[5131] 218666_T2 torch.Size([46, 3, 448, 448]) → T0:0.007(y=0) | T1:0.009(y=0) | T2:0.014(y=0) | T3:0.017(y=0)
   running: T0 acc=0.99 auc=0.635; T1 acc=0.98 auc=0.669; T2 acc=0.97 auc=0.641; T3 acc=0.96 auc=0.662; T4 acc=0.93 auc=0.667; T5 acc=0.88 auc=0.687 

[5132] 218675_T2 torch.Size([73, 3, 448, 448]) → T0:0.010(y=0) | T1:0.016(y=0)
   running: T0 acc=0.99 auc=0.635; T1 acc=0.98 auc=0.669; T2 acc=0.97 auc=0.641; T3 acc=0.96 auc=0.662; T4 acc=0.93 auc=0.667; T5 acc=0

,row,patient_id,logit_t0,prob_t0,target_t0,mask_t0,logit_t1,prob_t1,target_t1,mask_t1,...,target_t3,mask_t3,logit_t4,prob_t4,target_t4,mask_t4,logit_t5,prob_t5,target_t5,mask_t5
0,0,121403_T0,-4.812308,0.008064,0.0,True,-4.287579,0.013552,0.0,True,...,0.0,True,-3.841652,0.021007,0.0,True,-2.639340,0.066649,-1.0,False
1,1,203699_T0,-4.595718,0.009994,0.0,True,-4.161450,0.015346,0.0,True,...,0.0,True,-3.790897,0.022077,0.0,True,-2.535252,0.073424,-1.0,False
2,2,118635_T0,-5.120705,0.005936,0.0,True,-4.722454,0.008815,-1.0,False,...,-1.0,False,-3.935210,0.019167,-1.0,False,-2.858202,0.054259,-1.0,False
3,3,103460_T0,-4.134613,0.015757,0.0,True,-3.829964,0.021249,0.0,True,...,0.0,True,-3.339987,0.034225,-1.0,False,-2.243493,0.095912,-1.0,False
4,4,125587_T0,-4.476805,0.011242,0.0,True,-4.113390,0.016089,0.0,True,...,0.0,True,-3.658497,0.025124,0.0,True,-2.627770,0.067372,0.0,True
5,5,109518_T0,-5.240762,0.005268,0.0,True,-4.826488,0.007951,0.0,True,...,-1.0,False,-4.243279,0.014157,-1.0,False,-3.235594,0.037848,-1.0,False
6,6,201134_T0,-4.950307,0.007031,0.0,True,-4.650550,0.009466,0.0,True,...,0.0,True,-4.144935,0.015597,-1.0,False,-3.165404,0.040489,-1.0,False
7,7,207956_T0,-3.934309,0.019184,0.0,True,-3.541873,0.028144,0.0,True,...,0.0,True,-3.215008,0.038605,0.0,True,-2.406022,0.082715,-1.0,False
8,8,133072_T0,-5.127680,0.005895,0.0,True,-4.585989,0.010091,0.0,True,...,-1.0,False,-4.007025,0.017863,-1.0,False,-2.695488,0.063240,-1.0,False
9,9,101417_T0,-3.872192,0.020388,0.0,True,-3.333765,0.034431,0.0,True,...,0.0,True,-2.639888,0.066615,-1.0,False,-1.520543,0.179382,-1.0,False


9 · (OPTIONAL) Inspect a few predictions

In [10]:
import matplotlib.pyplot as plt

patient_idx = 0
probs = 1/(1+np.exp(-logits[patient_idx]))   # sigmoid
for t, p in enumerate(probs):
    print(f"Patient {patient_idx} - task{t}: prob={p:.3f} | target={targets[patient_idx,t]} | mask={masks[patient_idx,t]}")


Matplotlib created a temporary cache directory at /tmp/matplotlib-b2z6ct0c because the default path (/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


NameError: name 'logits' is not defined

Troubleshooting checklist
Key mismatch? - double-check that the num_attn_heads, num_layers, and dropout_rate you pass in exactly match those used when training the saved checkpoint.

CUDA OOM? - increase chunk_depth, lower out_size, or export an inference-only version of the model that processes fewer chunks.

No test data found? - ensure the CSV actually contains rows with split == "test" and non-null file_path.